In [10]:
import numpy as np
import time

# Monte Carlo parameters
num_simulations = 100000  # Initial number of simulations
tolerance = 0.01  # 1% relative error tolerance

# Model parameters
T = np.float128(1.0)  # Time to maturity
n_steps = 100  # Number of time steps
dt = T / n_steps
H = np.float128(0.1)  # Hurst parameter
rho = np.float128(-0.7)  # Correlation coefficient
v0 = np.float128(0.04)  # Initial forward variance
eta = np.float128(1.9)  # Volatility of volatility

# Time grid
times = np.linspace(0, T, n_steps + 1, dtype=np.float128)

def generate_brownian_motion(n_paths, n_steps, dt):
    return np.cumsum(np.sqrt(dt, dtype=np.float128) * np.random.randn(n_paths, n_steps).astype(np.float128), axis=1, dtype=np.float128)

def simulate_rough_bergomi(n_paths):
    dW1 = generate_brownian_motion(n_paths, n_steps, dt)
    dW2 = rho * dW1 + np.sqrt(np.float128(1) - rho**2, dtype=np.float128) * generate_brownian_motion(n_paths, n_steps, dt)
    
    logV = np.zeros((n_paths, n_steps + 1), dtype=np.float128)
    logV[:, 0] = np.log(v0, dtype=np.float128)
    for t in range(1, n_steps + 1):
        logV[:, t] = logV[:, t-1] + eta * np.sqrt(2 * H, dtype=np.float128) * dW2[:, t-1] * (times[t] - times[t-1])**(H - np.float128(0.5))
    
    V = np.exp(logV, dtype=np.float128)
    
    logS = np.zeros((n_paths, n_steps + 1), dtype=np.float128)
    logS[:, 0] = np.float128(0.0)
    for t in range(1, n_steps + 1):
        logS[:, t] = logS[:, t-1] - np.float128(0.5) * V[:, t-1] * dt + np.sqrt(np.maximum(V[:, t-1], np.float128(0.0), dtype=np.float128) * dt, dtype=np.float128) * dW1[:, t-1]
    
    S = np.exp(logS, dtype=np.float128)
    
    return S[:, -1]

def monte_carlo_simulation(target_error=0.01):
    np.random.seed(42)
    start_time = time.time()
    
    n_paths = num_simulations
    relative_error = np.inf
    current_mean = np.float128(0.0)
    current_variance = np.float128(0.0)
    iteration = 0
    
    while relative_error > target_error:
        iteration += 1
        S_T = simulate_rough_bergomi(n_paths)
        current_mean = np.mean(S_T, dtype=np.float128)
        current_variance = np.var(S_T, dtype=np.float128)
        
        # Calculate the relative error
        standard_error = np.sqrt(current_variance / n_paths, dtype=np.float128)
        relative_error = standard_error / current_mean
        
        # Double the number of paths for the next iteration
        n_paths *= 2
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    
    print(f"Converged in {iteration} iterations")
    print(f"Estimated price: {current_mean}")
    print(f"Relative error: {relative_error:.5f}")
    print(f"Elapsed time: {elapsed_time:.2f} seconds")
    
    return current_mean, relative_error, elapsed_time

# Running the simulation
price, error, elapsed_time = monte_carlo_simulation()

# Print results
print(f"Final price: {price:.4f}")
print(f"Final relative error: {error:.4f}")
print(f"Total elapsed time: {elapsed_time:.2f} seconds")




: 